In [1]:
import os
import cv2
from keras.preprocessing import image

In [2]:
categories = ['with_mask','without_mask']

In [3]:
data = []
for category in categories:
    path = os.path.join('mask-detection-data',category)
    label = categories.index(category)
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        data.append([img,label])

In [4]:
len(data)

1006

In [5]:
import random

In [6]:
random.shuffle(data)

In [7]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

In [8]:
import numpy as np

In [9]:
X=np.array(X)
y = np.array(y)

In [10]:
X.shape

(1006, 224, 224, 3)

In [11]:
y.shape

(1006,)

In [12]:
X = X/255

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [14]:
from keras.applications.vgg16 import VGG16

In [15]:
vgg = VGG16()

In [16]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
from keras import Sequential

In [18]:
model = Sequential()

In [19]:
for layers in vgg.layers[:-1]:
    model.add(layers)

In [20]:
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [21]:
from keras.layers import Dense

In [22]:
model.add(Dense(1,activation='sigmoid'))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [24]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
26/26 [==============================] - 100s 4s/step - loss: 0.6432 - accuracy: 0.6853 - val_loss: 0.5279 - val_accuracy: 0.7574
Epoch 2/5
26/26 [==============================] - 97s 4s/step - loss: 0.4198 - accuracy: 0.8769 - val_loss: 0.3256 - val_accuracy: 0.9356
Epoch 3/5
26/26 [==============================] - 93s 4s/step - loss: 0.3344 - accuracy: 0.8905 - val_loss: 0.2720 - val_accuracy: 0.9554
Epoch 4/5
26/26 [==============================] - 94s 4s/step - loss: 0.2748 - accuracy: 0.9254 - val_loss: 0.2449 - val_accuracy: 0.9554
Epoch 5/5
26/26 [==============================] - 91s 4s/step - loss: 0.2447 - accuracy: 0.9353 - val_loss: 0.2062 - val_accuracy: 0.9455


In [26]:
cap = cv2.VideoCapture(0)

In [34]:
while True:
    ret,frame = cap.read()
    frame = cv2.resize(frame,(224,224))
    y_pred = detect_face_mask(frame)
    draw_label(frame,y_pred,(30,30),(0,255,0))
    cv2.imshow('window',frame)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cv2.destroyAllWindows()

1/1 [==============================] - 0s 169ms/step


KeyboardInterrupt: 

In [28]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))

    if y_pred > 0.5:
        return "no mask"
    else:
        return "mask"

In [33]:
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_ITALIC,1,cv2.FILLED)
    end_x = pos[0]+text_size[0][0]+2
    end_y = pos[1]+text_size[0][1]-2

    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_ITALIC,1,(0,0,0),cv2.LINE_AA)